# Project 4

# Define lexnode

In [195]:
from collections import Counter

class LexNode:
    def __init__(self, char, kid = None, parent = None, isLastSame = False, isLast = False, isChild = False):
        self.char = char
        self.children = []
        self.property = 0
        self.idx = 0
        self.kid = kid
        self.parent = parent
        self.isLastSame = isLastSame
        self.isLast = isLast
        self.isChild = isChild

    def get_max_level(self, level = 0):
        if len(self.children) == 0:
            return level
        child_levels = [child.get_max_level(level = level + 1) for child in self.children]
        return max(child_levels)
  
class LexTree:
  def __init__(self, words):
    max_length = max(len(word) for word in words)
    self.words_len = [len(word) for word in words]
    padded_words = [word.ljust(max_length, "0") for word in words]
    self.words = padded_words
    self.root_node = LexNode('*')
    self.root_node.property = 1
    self.word_max_len = max_length
  
  def convert_to_lex(self):
    word_list_lex = []
    for word in self.words:
        lex_for_each_word = []
        for character in word:
            char = LexNode(character)
            lex_for_each_word.append(char)
        word_list_lex.append(lex_for_each_word)
    return word_list_lex

  def LexTree_build(self):
    word_list_lex = self.convert_to_lex()
    for row in range(len(word_list_lex)):
      for column in range(len(word_list_lex[0])):
        if word_list_lex[row][column].char == "0":
          break
        elif row == 0:
          current_node = word_list_lex[row][column]
          if column == self.words_len[row] - 1:
            current_node.isLast = True
        elif column == 0:
          pass
        else:
          previous_node=word_list_lex[row][column - 1]
          current_node=word_list_lex[row][column]
          compare_node=word_list_lex[row - 1][column]
          if current_node.char == compare_node.char:
            continue
          else:
            previous_node.isLastSame = True
            parent_node = word_list_lex[row - 1][column - 1]
            parent_node.kid = previous_node
            previous_node.parent = parent_node
            previous_node.isChild = True
            break
    
    for row in range(len(word_list_lex)):
      for column in range(len(word_list_lex[0])):
        current_node = word_list_lex[row][column]
        if column == self.words_len[row] - 1:
          current_node.isLast = True
    for row in range(len(word_list_lex)):
      for column in range(len(word_list_lex[0])):
        if row >= 1 and column >= 1:
          previous_node = word_list_lex[row][column - 1]
          current_node = word_list_lex[row][column]
          parent_node = word_list_lex[row - 1][column]
          if parent_node.isLast and current_node.isLastSame:
            previous_node.isChild = True
            current_node.isChild = False
            parent_node = word_list_lex[row-1][column-1]
            parent_node.kid = previous_node
            previous_node.parent = parent_node
    c = [0]
    for row in range(len(word_list_lex)):
      for column in range(len(word_list_lex[0])):
        current_node=word_list_lex[row][column]
        if current_node.isChild:
          c.append(column)
    d = []
    for row in range(len(word_list_lex)):
      e = []
      for column in range(len(word_list_lex[0])):
        current_node = word_list_lex[row][column]
        e.append(current_node.isLast)
      d.append(e)
    for row in range(len(word_list_lex)):
      str = ""
      for column in range(len(word_list_lex[0])):
        current_node = word_list_lex[row][column]
        if current_node.char == "0":
            break
        elif column == c[row]:
          if row == 0:
            str = str + current_node.char + "--"
          else:
            str = str + "|--"
        elif column < c[row] :
          str = str + "|  "
        elif column > c[row]:
          if not current_node.isLast:
            str = str + current_node.char + "--"
          else:
            str = str + current_node.char 
      print(str)
    return word_list_lex,c
  
  def add_lex_node(self, parent_node, child_node):
        assert isinstance(parent_node, LexNode) and isinstance(child_node, LexNode), "Both parent and child must be LexNode instances"
        parent_node.children.append(child_node)
    
  def construct_lexicon_tree(self):
        self._recursive_tree_build(self.root_node, self.words_list, 0)
        
  def _recursive_tree_build(self, current_node, remaining_words, current_index):
        if current_index == self.longest_word_length:
            print("Finished")
        words_ending_here = []
        for word in remaining_words:
            if (current_index + 1 < self.longest_word_length and word[current_index + 1] == '$') or current_index == self.longest_word_length - 1:
                end_node = LexNode(word[current_index])
                end_node.property = 2
                end_node.children.append(self.root_node)
                self.add_lex_node(current_node, end_node)
                words_ending_here.append(word)
        for word in words_ending_here:
            remaining_words.remove(word)
        remaining_word_count = len(remaining_words)
        next_chars = [remaining_words[idx][current_index] for idx in range(remaining_word_count)]
        char_counts = Counter(next_chars)
        for char, count in char_counts.items():
            next_node = LexNode(char)
            self.add_lex_node(current_node, next_node)
            words_for_next_char = [word for word in remaining_words if word[current_index] == char]
            self._recursive_tree_build(next_node, words_for_next_char, current_index + 1)

def find_parent_row(array):
  parents = []
  previous_value = None
  for i, value in enumerate(array):
    if value != previous_value:
      if i == 0:
        parents.append(i)
      else:
        parents.append(i - 1)
      previous_value = value
  return parents

def read_words_from_file(file_path):
    
    words = []
    try:
        with open(file_path, 'r', encoding = 'utf-8') as file:  
            for line in file:
                words_in_line = line.strip().split()
                words.extend(["*" + word for word in words_in_line])
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding = 'windows-1252') as file: 
            for line in file:
                words_in_line = line.strip().split()
                words.extend(["*" + word for word in words_in_line])
    return words

file_path = 'dict_1.txt'
word_list = read_words_from_file(file_path)
print(word_list)
Tree = LexTree(word_list)
lex_matrix,c = Tree.LexTree_build()
parents_row = find_parent_row(c)
dict_words = []


['*a', '*abandon', '*abbas', '*abbreviation', '*abdominal', '*abela', '*abernethy', '*abides', '*able', '*abolishing', '*abortionists', '*about', '*abraham', '*abridge', '*absences', '*absolved', '*abstinent', '*abundantly', '*accedes', '*accentuating', '*accept', '*accident', '*accommodated', '*accompany', '*accomplishment', '*accord', '*accountancy', '*accrue', '*accuride', '*acero', '*aches', "*acker's", '*ackroyd', '*acquaint', '*acquit', '*acronym', '*across', '*activate', "*actor's", '*actually', '*acumen', '*adachi', '*adaptec', '*addison', '*adelsberger', '*adirondack', '*adjuncts', '*admirable', '*admits', '*adolf', '*adorabelle', '*adrenaline', '*adult', '*advantage', '*adversaries', '*advice', '*advil', '*advised', '*advocate', '*aeronautical', '*aesthete', '*affairs', '*affirmations', '*affluent', "*afghanistan's", '*africa', '*after', '*aftereffect', '*again', '*against', '*aggregates', '*agility', '*agonizes', '*agreed', '*agreeing', '*agreement', '*agrippa', '*aguilera',

# Build lextree

In [196]:
buildlextree=LexTree(word_list)
buildlextree.construct_lexicon_tree
lextree=buildlextree.root_node

In [197]:
import numpy as np
import copy
class SpellChecker:
    def __init__(self, beam, longest_word_lenth = 45, tolerate_num_of_error = 2):
        self.beam = int(beam)
        self.lextree = None
        self.dist_fun = lambda *args: int(args[0] != args[1])
        self.longest_word_lenth = longest_word_lenth
        self.tolerate_num_of_error = tolerate_num_of_error
        
    def fit(self, lextree):
        self.lextree = lextree
        assert type(self.lextree) is LexNode
        self.nodes = []
        self.get_nodes(self.lextree)
        self.transitions = {}
        self.get_children = {}
        n_nodes = len(self.nodes)
        self.word_ends = []
        for i in range(n_nodes):
            n = self.nodes[i]
            if n.property == 2:
                self.word_ends.append(i)
            self.get_children[i]=[]
            if len(n.children) > 0:
                for child in n.children:
                    self.get_children[i].append(self.nodes.index(child))
                    self.transitions[self.nodes.index(child)] = i
        self.transitions[0] = 0 

    def mute_nodes(self, r):
        self.muted_nodes[r] = 1
        to_mute_children = self.get_children[r]
        if len(to_mute_children) != 0:
            for child_idx in to_mute_children:
                if child_idx != 0:
                    self.mute_nodes(child_idx)
                
    def get_nodes(self, lexnode):
        self.nodes.append(lexnode)
        if lexnode.property == 2:
            return
        for child in lexnode.children:
            self.get_nodes(child)
    
    def text_viterbi(self, x, longest = True, get_string = True):
        x = '*' + x
        n_cols = len(x)
        n_rows = len(self.nodes)
        costs = np.full((n_rows, n_cols), np.inf)
        costs[0, :] = range(n_cols)
        n_nodes = len(self.nodes)
        self.muted_nodes = np.zeros([n_nodes])

        for c in range(n_cols):
            for r in range(1, n_rows):
                if not self.muted_nodes[r]:
                    current_cost = self.dist_fun(self.nodes[r].val, x[c])
                    if c == 0:
                        if self.transitions[r] == 0:
                            costs[r,c]=current_cost
                        else:
                            
                            costs[r,c]=costs[self.transitions[r],c]+current_cost
                    elif c==1:
                        if current_cost==0:
                            to_check=[costs[r,c-1]+1, costs[self.transitions[r],c-1]] 
                        else:
                            to_check=[costs[r,c-1], costs[self.transitions[r],c-1]] 
                        nnn=np.argmin(to_check)
                        costs[r,c]=to_check[nnn]+current_cost
                    else:
                        if current_cost==0:
                            to_check=[costs[r,c-1]+1, costs[self.transitions[r],c-1], costs[self.transitions[r],c-2]+1]
                        else:
                            to_check=[costs[r,c-1], costs[self.transitions[r],c-1], costs[self.transitions[r],c], costs[self.transitions[r],c-2]+1]
                        nnn=np.argmin(to_check)
                        costs[r,c]=to_check[nnn]+current_cost
                        
                    if costs[r,c]>=self.tolerate_num_of_error+len(x):
                        self.mute_nodes(r)
            
        costs_for_words=[]
        for end_idx in self.word_ends:
            costs_for_words.append(costs[end_idx,-1])
        
        min_cost=min(costs_for_words)
        min_idxs=np.where(costs_for_words==min_cost)[0]
        
        if get_string:
            result=self.nodes_to_word(min_idxs,longest=longest)
            return result
        else:
            return min_idxs, min_cost
    
    def nodes_to_word(self,nodes,longest=True):
        results=[]
        for idx in nodes:
            
            idx=self.word_ends[idx]
            
            result=""
            while self.transitions[idx]!=0:
                result=self.nodes[idx].val+result
                idx=self.transitions[idx]
            result=self.nodes[idx].val+result
            results.append(result)
            
        if longest:
            res = max(results, key=len, default='')
            return res
        return results
    
    
    def get_nodes_from_token(self,tokens,end_token):
        nodes=[]
        time=tokens[0]
        value=tokens[1]
        previous=tokens[2]
        previous_time=end_token[-1]
        nodes.append(end_token[1])
        while previous_time!=0:
            idxs=np.where(time==previous_time)[0]
            previous_time=previous[idxs[0]]
            nodes.insert(0,value[idxs[0]])
        return nodes
    
    def nodes_to_sentence(self,nodes):
        
        sentence=""
        for idx in nodes:
            
            result=""
            while self.transitions[idx]!=0:
                result=self.nodes[idx].val+result
                idx=self.transitions[idx]
            result=self.nodes[idx].val+result
            sentence+=result+" "
        
        return  sentence
    
    def nodes_list_to_sentences(self,nodes_list):
        words_list=self.nodes_list_to_words(nodes_list)
        num_words_in_each_words_list=list(map(len, words_list))
        num_possible_sentences=np.product(num_words_in_each_words_list)
        print("there are {} possible sentences".format(num_possible_sentences))
        padded_words_list=[]
        for words in words_list:
            if len(words)==0:
                continue
            repeat_time=int(num_possible_sentences/len(words))
            aaa_words_list=[]
            for i in range(repeat_time):
                aaa_words_list+=words
            padded_words_list.append(aaa_words_list)
        
        padded_words_list=np.array(padded_words_list)
        padded_words_list=np.column_stack(padded_words_list)
        sentences=[]
        for i in padded_words_list:
            sentence=""
            for word in i:
                sentence+=word +" "
            sentences.append(sentence)
        return sentences
    
    def nodes_list_to_words(self,nodes_list):
        all_words=[]
        for current_word_idxs in nodes_list:
            if len(current_word_idxs)==0:
                continue
            else:
                current_words=[]
                for idx in current_word_idxs:
                    
                    result=""
                    while self.transitions[idx]!=0:
                        result=self.nodes[idx].val+result
                        
                        idx=self.transitions[idx]
                    result=self.nodes[idx].val+result
                    current_words.append(result)
                all_words.append(current_words)
        
        return all_words
    
    def check_end_of_words(self,costs):
        costs_for_words=[]
        for end_idx in self.word_ends:
            costs_for_words.append(costs[end_idx])
        min_cost=min(costs_for_words)
        min_idxs=np.where(costs_for_words==min_cost)[0]
        min_idxs = min_idxs[min_idxs != 0]
        return min_cost,min_idxs
    
    def unsegment_text_viterbi():
        pass
    
    

# Initialize SpellChecker

In [198]:
spellchecker=SpellChecker(3,longest_word_lenth=9)
spellchecker.fit(lextree)

In [199]:
typos_with_beam=['once upon a time wigle uncharacteristically as ing of uncharacteristically the uncharacteristically kamen to life at the foot of he uncharacteristically as a uncharacteristically he grew uncharacteristically and uncharacteristically big of uncharacteristically well to do and lived by a uncharacteristically of the uncharacteristically uncharacteristically in a uncharacteristically uncharacteristically now at that them uncharacteristically was a uncharacteristically uncharacteristically in the uncharacteristically the uncharacteristically uncharacteristically saw the uncharacteristically uncharacteristically of the uncharacteristically and she uncharacteristically a uncharacteristically to eke as uncharacteristically so she seda to her lord serb i uncharacteristically to feet the uncharacteristically of that uncharacteristically king of the uncharacteristically ', 'dodd life sadye the uncharacteristically i leed in the uncharacteristically and heed uncharacteristically on dry land how can we mach him ', 'dyk hua or by crock see uncharacteristically he met be kort if i uncharacteristically get heed i uncharacteristically die ', 'all rate uncharacteristically the uncharacteristically uncharacteristically ohr uncharacteristically uncharacteristically uncharacteristically i have a plan i wild give coo his cart to feet ', 'so when the uncharacteristically was uncharacteristically on the bank of the uncharacteristically after uncharacteristically a drink of water the uncharacteristically dromon near and seide sir uncharacteristically what do your live on bad uncharacteristically in this nolde uncharacteristically uncharacteristically on the uncharacteristically side of the uncharacteristically uncharacteristically is no end to the uncharacteristically uncharacteristically and uncharacteristically uncharacteristically wit uncharacteristically uncharacteristically as coney is it not bear to kroc uncharacteristically lande have allen uncharacteristically of uncharacteristically uncharacteristically to mate ', 'lore uncharacteristically the uncharacteristically uncharacteristically the uncharacteristically is uncharacteristically and uncharacteristically how shall i at uncharacteristically ', 'life coo want to gob i fill let ju sit upon my back and kady you over ', 'the uncharacteristically uncharacteristically am and uncharacteristically come here then seide the uncharacteristically up on myer back with coo and up the uncharacteristically uncharacteristically but when the uncharacteristically had swum a uncharacteristically wave he uncharacteristically the uncharacteristically under the uncharacteristically ', 'good uncharacteristically you are uncharacteristically me uncharacteristically uncharacteristically the uncharacteristically what is that or ', 'the uncharacteristically said you uncharacteristically i am uncharacteristically your out of uncharacteristically uncharacteristically uncharacteristically not a bit of it my wife has a uncharacteristically for uncharacteristically uncharacteristically and i uncharacteristically to eve it to ohr to mate ', 'uncharacteristically said the uncharacteristically it is nice of coo to tell me what if our cart uncharacteristically uncharacteristically us when we go uncharacteristically uncharacteristically the tried tops it wild be all uncharacteristically to uncharacteristically ', 'all where do you keep it asked the uncharacteristically ', 'the uncharacteristically uncharacteristically out a big tried with uncharacteristically of crye uncharacteristically uncharacteristically not far off side uncharacteristically he uncharacteristically are our uncharacteristically uncharacteristically on uncharacteristically fine tried ', 'if you uncharacteristically uncharacteristically me your uncharacteristically said the uncharacteristically then i uncharacteristically kill gob ', 'uncharacteristically meet to the uncharacteristically then and i all uncharacteristically it out to your ', 'the uncharacteristically uncharacteristically him to the uncharacteristically the uncharacteristically uncharacteristically off his back and uncharacteristically hupe the nigg tree sat uncharacteristically it oh uncharacteristically uncharacteristically uncharacteristically he you uncharacteristically that uncharacteristically were uncharacteristically that kept uncharacteristically uncharacteristically in a uncharacteristically you are a uncharacteristically and i have uncharacteristically you you may kept your uncharacteristically to uncharacteristically lore body is uncharacteristically but you have no uncharacteristically ', 'and uncharacteristically to uncharacteristically this uncharacteristically he uncharacteristically the uncharacteristically uncharacteristically ', 'uncharacteristically uncharacteristically uncharacteristically tops uncharacteristically the water uncharacteristically i see ', 'uncharacteristically of them i ant them not my nigg is uncharacteristically uncharacteristically for me ', 'uncharacteristically is yukon uncharacteristically uncharacteristically buts how uncharacteristically uncharacteristically is you with ', 'now go uncharacteristically ways serb uncharacteristically for i eve had the uncharacteristically how with ', 'the uncharacteristically uncharacteristically as said and uncharacteristically as if he had lost a uncharacteristically uncharacteristically of uncharacteristically ant uncharacteristically uncharacteristically to the uncharacteristically where he lived ']
corrected_withbeam=''
for x in typos_with_beam:
    corrected_withbeam+=x
    
corrected_withbeam

'once upon a time wigle uncharacteristically as ing of uncharacteristically the uncharacteristically kamen to life at the foot of he uncharacteristically as a uncharacteristically he grew uncharacteristically and uncharacteristically big of uncharacteristically well to do and lived by a uncharacteristically of the uncharacteristically uncharacteristically in a uncharacteristically uncharacteristically now at that them uncharacteristically was a uncharacteristically uncharacteristically in the uncharacteristically the uncharacteristically uncharacteristically saw the uncharacteristically uncharacteristically of the uncharacteristically and she uncharacteristically a uncharacteristically to eke as uncharacteristically so she seda to her lord serb i uncharacteristically to feet the uncharacteristically of that uncharacteristically king of the uncharacteristically dodd life sadye the uncharacteristically i leed in the uncharacteristically and heed uncharacteristically on dry land how can w

In [200]:
f=open("storycorrect.txt")
lines = f.readlines()
temp=""
for line in lines:
    temp+=line
f.close()

temp

'Once upon a time, while Brahmadatta was king of Benares, the Bodhisatta came to life at the foot of the Himalayas as a monkey. He grew strong and sturdy, big of frame, well to do, and lived by a curve of the river Ganges in a forest haunt. Now at that time there was a crocodile dwelling in the Ganges. The crocodile\'s mate saw the great frame of the monkey, and she conceived a longing to eat his heart. So she said to her lord, "Sir, I desire to eat the heart of that great king of the monkeys!"\n\n"Good wife," said the crocodile, "I live in the water and he lives on dry land. How can we catch him?"\n\n"By hook or by crook," she replied, "he must be caught. If I don\'t get him, I shall die."\n\n"All right," answered the crocodile, consoling her, "don\'t trouble yourself. I have a plan. I will give you his heart to eat."\n\nSo when the Bodhisatta was sitting on the bank of the Ganges, after taking a drink of water, the crocodile drew near, and said, "Sir Monkey, why do you live on bad fr

# Accuracy

In [201]:
def distance(x, y):
    return int(x!=y)

def acc_checker1(inp, tem, distance):
    M, N=len(tem), len(inp)
    cost=np.zeros((M, N))
    
    cost[0, 0] = distance(inp[0], tem[0])
    for i in range(1, M):
        cost[i, 0] = cost[i-1, 0] + distance(inp[0], tem[i])

    for j in range(1, N):
        cost[0, j] = cost[0, j-1] + distance(inp[j], tem[0])

    for i in range(1, N):
        for j in range(1, M):
        
            choices = cost[j, i-1], cost[j-1, i-1], cost[j-1, i]
            cost[j, i] = min(choices) + distance(inp[i], tem[j])

    return cost[-1, -1]


In [202]:
cost_withbeam=acc_checker1(corrected_withbeam, temp, distance)
cost_withbeam

3237.0

In [203]:
acc_withbeam=1-cost_withbeam/len(corrected_withbeam)
print('Accuracy with beam 3: ', acc_withbeam)

Accuracy with beam 3:  0.40364775239498896


## Segmentation

In [204]:
n_best=10
file=open("unsegmented0.txt","r").readlines()
total_cost=0
for sentence in file:
    sentence=sentence.strip('\n')
    min_sentence_cost=spellchecker.unsegment_text_viterbi(sentence,n_best=n_best,get_sentence=False,loop_cost=0.00001)
    print("the min cost for this sentence is {}".format(min_sentence_cost))
    total_cost+=min_sentence_cost
print("Over all cost for this file with n_best equal to {} is {}".format(n_best,total_cost))

TypeError: SpellChecker.unsegment_text_viterbi() got an unexpected keyword argument 'n_best'

In [ ]:
total_cost

inf

In [ ]:
n_best=10

file=open("unsegmented.txt","r").readlines()
total_cost=0
for sentence in file:
    sentence=sentence.strip('\n')
    min_sentence_cost=spellchecker.unsegment_text_viterbi(sentence,n_best=n_best,get_sentence=False,loop_cost=0.00001)
    print("the min cost for this sentence is {}".format(min_sentence_cost))
    total_cost+=min_sentence_cost
print("Over all cost for this file with n_best equal to {} is {}".format(n_best,total_cost))

1 th input character
inf
2 th input character
inf
3 th input character
inf
4 th input character
inf
5 th input character
inf
6 th input character
inf
7 th input character
inf
8 th input character
inf
9 th input character
inf
10 th input character
inf
11 th input character
inf
12 th input character
inf
13 th input character
inf
14 th input character
inf
15 th input character
inf
16 th input character
inf
17 th input character
inf
18 th input character
inf
19 th input character
inf
20 th input character
inf
21 th input character
inf
22 th input character
inf
23 th input character
inf
24 th input character
inf
25 th input character
inf
26 th input character
inf
27 th input character
inf
28 th input character
inf
29 th input character
inf
30 th input character
inf
31 th input character
inf
32 th input character
inf
33 th input character
inf
34 th input character
inf
35 th input character
inf
36 th input character
inf
37 th input character
inf
38 th input character
inf
39 th input character

/var/folders/7w/hyls2fc56q36tpl1bsjwx6340000gn/T/ipykernel_5373/4009524958.py:268: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not temp_cost_column[self.word_ends]:


In [ ]:
total_cost

inf